In [4]:
from lib.graph.graph.graph import Graph
from lib.gnnanalyzer.gnnanalyzer.localgnnanalyzer import LocalGNNAnalyzer
from lib.gnnanalyzer.gnnanalyzer.localimpactcalculator import (
    LocalImpactCalculator,
    LocalImpactCalculationMethod,
)

In [2]:
g = Graph()
g.import_edges_from_edge_list(
    data="./datasets/cora/cora.cites",
    source_target_col=(0, 1),
    override_data_file_extension=".txt",
    delimiter="	",
)

In [5]:
impact = LocalImpactCalculator(method=LocalImpactCalculationMethod.ABSOLUTE_DIFFERENCE)
analyzer = LocalGNNAnalyzer(graph=g, local_impact_calculator=impact)